![Py4Eng](img/logo.png)

# Graphical user interface
## Yoav Ram

# *Tk* GUI

We will be using [*Tk*](http://www.tkdocs.com/), a user interface toolkit that makes it easy to build desktop graphical user interfaces. Tk is cross-platform and it plays nicely with *Matplotlib* and with the Jupyter notebook.

We don't need to install anything, because Tk is packaged with Python, and is accessible via the `tkinter` module which is part of the standard library.

## Hello World!

Let's start with a simple *Hello World!* application - just a window with a button; when the button is clicked, an *Hello World!* dialog comes up. This example follows [Learning IPython for Interactive Computing and Data Visualization](http://ipython-books.github.io/minibook/) by Cyrille Rossant, pg. 85, but modifed to use Tk.

We use the notebook's magic command `%gui` to let the notebook know that we are using Tk, then import the `tkinter` module - note that in Python 2 this module was called `Tkinter`. We also import `tkinter.messagebox`, which has helper functions to easily create... message boxes.

In [1]:
%gui tk
import tkinter
import tkinter.messagebox

Next, we define out main application window, a class we call `HelloWorld`. We add a push-button, with the label `Click me`, and connect it to the method `clicked`. 

We then create a simple layout and show the window (since it's the main window of the application). 

The `clicked` method creates a dialog with an `OK` button which says `Hello World!`.

Finally, we create the Tk application and the main window. Usually we would need to call `app.master.mainloop()` to start the application mainloop, but Jupyter takes care of that because we called `%gui tk`.

In [6]:
class HelloWorld:
    def __init__(self):
        self.master = master = tkinter.Tk()
        self.frame = tkinter.Frame(master)
        self.frame.pack()
        
        self.button = tkinter.Button(self.frame)
        # the following could have been given in the init above if your prefer
        self.button["text"] = "Click me"
        self.button["command"] = self.clicked
        self.button.pack()

    def clicked(self):
        tkinter.messagebox.showinfo("Hello world", "You clicked a button.")        

app = HelloWorld()

Interestingly, starting the GUI **doesn't block** the notebook (you can see the empty rather than filled circle at the top right of the notebook) which means we can interact with our window through the notebook. This is very useful for testing and debugging.

For example, we can trigger the `clicked` method without actually clicking the button:

In [7]:
app.clicked()

Change the window title and size:

In [8]:
app.master.geometry('500x50')
app.master.title("New title")

''

And close the window:

In [9]:
app.master.destroy()

## PyGubu Designer

When creating more sophisticated application, it's more convinient to work with a designer - a WYSIWYG GUI editor. 

[Pygubu](https://github.com/alejandroautalan/pygubu) is such a tool, allowing us to create and edit *XML* files that have an `.ui` extension and define the layout and design of a *Tk* GUI application and then load these XML files from our Python model code to use the design.

Let's do a simple example before diving into a more sophisticated example. Install *Pygubu* (`conda run -n Py4Eng XXX` runs `XXX` after activating `Py4Eng` as the Python environment):

In [ ]:
!pip install pygubu

Now open the *Pygubu* application on your desktop (the notebook is locked until you close *Pygubu*; if you want to use it in parallel then you should open it from the terminal):

In [8]:
!pygubu-designer

/bin/sh: pygubu-designer: command not found


*Pygubu* will open and we can start adding widgets to it and design our GUI:

![QtDesigner](img/Pygubu.png)

We'll build a simple app with just a big textbox to write text to and save\load to\from a file.

The design is implemented in `../scripts/notepad.ui` - you can open it in the *Pygubu* (File -> Open...).

![QtDesigner Notepad UI](img/PygubuNotepad.png)

To start a new design you add a `Frame` or `TopLevel` and then you add the widgets you want to it - all from the widget list on the left. Afterwards, you use the widget editor on the bottom to define the widgets' variable name (`id`) and properties such as text, font, and width. 

![QtDesigner Notepad UI](img/PygubuNotepadLayout.png)

Next, you switch to the `Layout` tab to define the layout of the widgets within the frame. The default layout uses a grid system in which you can set the row and column of every widget, their alignment within their grid cell (`sticky`), horizontal and vertical padding (`padx` and `pady`). You can also have one widget span multiple rows or columns using `rowspan` and `colspan` (like I did with the textbox).

We can save our design (File -> Save...) it's saved as an XML file:

In [10]:
%less ../scripts/notepad.ui

<?xml version='1.0' encoding='utf-8'?>
<interface>
  <object class="tk.Frame" id="mainwindow">
    <property name="container">false</property>
    <property name="height">200</property>
    <property name="width">200</property>
    <layout>
      <property name="column">0</property>
      <property name="propagate">True</property>
      <property name="row">0</property>
    </layout>
    <child>
      <object class="tk.Text" id="textEdit">
        <property name="height">20</property>
        <property name="takefocus">true</property>
        <property name="width">40</property>
        <property name="wrap">word</property>
        <layout>
          <property name="column">0</property>
          <property name="columnspan">6</property>
          <property name="padx">5</property>
          <property name="pady">10</property>
          <property name="propagate">True</property>
          <property name="row">0</property>
        </layout>
      </object>
    </child>
    <child>
      

Once we have a nice design `.ui` file, we can use it in our application code

In [11]:
import pygubu

class Notepad:
    def __init__(self): 
        self.master = master = tkinter.Tk()
        master.title('Notepad')
        # Create a pygubu builder
        self.builder = builder = pygubu.Builder()
        # Load design file
        builder.add_from_file('../scripts/notepad.ui')
        # Create the mainwindow widget using a master as parent
        self.mainwindow = builder.get_object('mainwindow', master)

app = Notepad()

ERROR:pygubu.builder.builderobject:Failed to set property 'container' on class '<class 'tkinter.Frame'>'. TclError: can't modify -container option after widget is created


Of course, to make this interactive we need to implement button callbacks and other UI logic.
We can define the button's callbacks in *Pygubu* (the `command` property), but I find it's better to separate design and functionality, so we will configure the callbacks from the code:

In [12]:
import pygubu

class Notepad:
    def __init__(self): 
        self.master = master = tkinter.Tk()
        master.title('Notepad')
        # Create a pygubu builder
        self.builder = builder = pygubu.Builder()
        # Load design file
        builder.add_from_file('../scripts/notepad.ui')
        # Create the mainwindow widget using a master as parent
        self.mainwindow = builder.get_object('mainwindow', master)
        # Get a button and set it's callback
        self.saveButton = builder.get_object('saveButton', master)
        self.saveButton["command"] = self.clicked

    def clicked(self):
        tkinter.messagebox.showinfo("Click", "You clicked a button.")
                
app = Notepad()

ERROR:pygubu.builder.builderobject:Failed to set property 'container' on class '<class 'tkinter.Frame'>'. TclError: can't modify -container option after widget is created


## Save button

We start with the save button. We need to:

- implement a new method, `save` that 
  - reads the text from the editor
  - reads the filename from the filename textbox
  - opens a file and write the text
  - catches exceptions and reports them with a dialog (like the first example in this session)
- connect the method to the button

In [13]:
class Notepad:
    def __init__(self): 
        self.master = master = tkinter.Tk()
        master.title('Notepad')
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.filenameEdit = builder.get_object('filenameEdit', master)
        self.saveButton = builder.get_object('saveButton', master)
        self.saveButton["command"] = self.save

    def save(self):
        text = self.textEdit.get(1.0, tkinter.END)
        filename = self.filenameEdit.get()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
        else:
            tkinter.messagebox.showinfo("Save", 
                    "Saved to {}".format(filename))
            
app = Notepad()

ERROR:pygubu.builder.builderobject:Failed to set property 'container' on class '<class 'tkinter.Frame'>'. TclError: can't modify -container option after widget is created


Note that while the window is open, you can introspect it in the notebook:

In [14]:
%less tmp.txt

bla bla



In [15]:
print(app.filenameEdit.get())

tmp.txt


In [16]:
fname = app.filenameEdit.get()
%less $fname

bla bla



## Load button

Next, we write a `load` method, which
- reads a filename from the filename textbox
- reads the text from the file
- puts the text in the editor
We then connect method to the load button.

In [17]:
class Notepad:
    def __init__(self): 
        self.master = master = tkinter.Tk()
        master.title('Notepad')
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.filenameEdit = builder.get_object('filenameEdit', master)
        self.saveButton = builder.get_object('saveButton', master)
        self.loadButton = builder.get_object('loadButton', master)
        self.saveButton["command"] = self.save
        self.loadButton["command"] = self.load

    def save(self):
        text = self.textEdit.get(1.0, tkinter.END)
        filename = self.filenameEdit.get()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
        else:
            tkinter.messagebox.showinfo("Save", 
                    "Saved to {}".format(filename))

    def load(self):
        filename = self.filenameEdit.get()
        try:
            with open(filename) as f:
                text = f.read()
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
        else:
            self.textEdit.delete(1.0, tkinter.END)
            self.textEdit.insert(1.0, text)

app = Notepad()

ERROR:pygubu.builder.builderobject:Failed to set property 'container' on class '<class 'tkinter.Frame'>'. TclError: can't modify -container option after widget is created


In [18]:
%pwd

'/Users/yoavram/Work/Teaching/Py4Eng/sessions'

In [20]:
print(app.textEdit.get(1.0, tkinter.END))

Hello Python!





## Browser button

Lastly, we will implement the browse button that will open a file dialog and save the result to the filename textbox. Let's first experiment:

In [21]:
import tkinter.filedialog
filename = tkinter.filedialog.askopenfilename(
    defaultextension='.txt', 
    filetypes=[('all files', '.*'), ('text files', '.txt')],
#     parent
    title="Choose file"
)
print(filename)

/Users/yoavram/Work/Teaching/Py4Eng/sessions/tmp.txt


In [22]:
import tkinter.filedialog
import pygubu

class Notepad:
    def __init__(self): 
        self.master = master = tkinter.Tk()
        master.title('Notepad')
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.filenameEdit = builder.get_object('filenameEdit', master)
        self.saveButton = builder.get_object('saveButton', master)
        self.loadButton = builder.get_object('loadButton', master)
        self.browseButton = builder.get_object('browseButton', master)
        self.saveButton["command"] = self.save
        self.loadButton["command"] = self.load
        self.browseButton["command"] = self.browse

    def save(self):
        text = self.textEdit.get(1.0, tkinter.END)
        filename = self.filenameEdit.get()
        try:
            with open(filename, 'w') as f:
                print(text, file=f)
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
        else:
            tkinter.messagebox.showinfo("Save", 
                    "Saved to {}".format(filename))

    def load(self):
        filename = self.filenameEdit.get()
        try:
            with open(filename) as f:
                text = f.read()
        except Exception as e:
            tkinter.messagebox.showinfo("Error", str(e))
        else:
            self.textEdit.delete(1.0, tkinter.END)
            self.textEdit.insert(1.0, text)

        
    def browse(self):
        filename = tkinter.filedialog.askopenfilename(
            defaultextension='.txt', 
            filetypes=[('all files', '.*'), ('text files', '.txt')],
            parent=self.mainwindow,
            title="Choose file"
        )
        self.filenameEdit.delete(0, tkinter.END)
        self.filenameEdit.insert(0, filename)
                
app = Notepad()

ERROR:pygubu.builder.builderobject:Failed to set property 'container' on class '<class 'tkinter.Frame'>'. TclError: can't modify -container option after widget is created


That's it, we have our notepad application.

## Exercise

Add a button called "All Caps" that changes the text to uppercase (using `str.upper()`).

# Scheduling and binding

Let's add a "Word Count" label that displays the number of words in the notepad. We easily write a method `update_word_count`

In [23]:
import pygubu

class Notepad:
    def __init__(self): 
        self.master = master = tkinter.Tk()
        master.title('Notepad')
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.counterLabel = builder.get_object('counterLabel', master)

    def update_word_count(self):
        text = self.textEdit.get(1.0, tkinter.END)
        words = text.split()
        word_count = len(words)
        self.counterLabel['text'] = '{:d}'.format(word_count)
                
app = Notepad()

ERROR:pygubu.builder.builderobject:Failed to set property 'container' on class '<class 'tkinter.Frame'>'. TclError: can't modify -container option after widget is created


We can update the word count label by calling `update_word_count`:

In [24]:
app.update_word_count()

However, we would like to do it automatically every second. The first instinct might be to use a thread, but threads cannot change widget; widgets must be changed from the mainloop (which Jupyter notebook starts for use, but with a regular application we would call `app.mainloop()` to start it). 

However, Tk has a good alternative - we can give a timed callback to the mainloop using `app.after`.

In [25]:
class Notepad:
    def __init__(self):
        self.master = master = tkinter.Tk()
        master.title('Notepad')
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.counterLabel = builder.get_object('counterLabel', master)

        self.update_word_count()
        
    ### methods removed for brevity, see sessions/gui.ipynb or scripts/notepad.py
    def update_word_count(self):
        text = self.textEdit.get(1.0, tkinter.END)
        words = text.split()
        word_count = len(words)
        self.counterLabel['text'] = '{:d}'.format(word_count)
        self.master.after(1000, self.update_word_count)

app = Notepad()

ERROR:pygubu.builder.builderobject:Failed to set property 'container' on class '<class 'tkinter.Frame'>'. TclError: can't modify -container option after widget is created


An alternative is to update the counter on every key stroke using by [binding](http://effbot.org/tkinterbook/tkinter-events-and-bindings.htm) `update_word_count` to key events on `textEdit`:

In [26]:
class Notepad:
    def __init__(self):
        self.master = master = tkinter.Tk()
        master.title('Notepad')
        self.builder = builder = pygubu.Builder()
        builder.add_from_file('../scripts/notepad.ui')
        self.mainwindow = builder.get_object('mainwindow', master)        
        self.mainwindow = builder.get_object('mainwindow', master)
        
        self.textEdit = builder.get_object('textEdit', master)
        self.counterLabel = builder.get_object('counterLabel', master)

        self.textEdit.bind('<Key>', self.update_word_count)
        
    ### methods removed for brevity, see sessions/gui.ipynb or scripts/notepad.py
    def update_word_count(self, *args):
        key = args[0].char
        if key in (' ', '\b', '\t'):
            text = self.textEdit.get(1.0, tkinter.END)
            words = text.split()
            word_count = len(words)
            self.counterLabel['text'] = '{:d}'.format(word_count)

        
app = Notepad()

ERROR:pygubu.builder.builderobject:Failed to set property 'container' on class '<class 'tkinter.Frame'>'. TclError: can't modify -container option after widget is created


# Tk and Matplotlib

[Matplotlib has support for Tk](http://matplotlib.org/examples/user_interfaces/embedding_in_tk.html), so that we can put a plot inside a Tk widget `QWidget`,

We have some importing to do. We first import `matplotlib` and set it to use the `TkAgg`. It's important to do this before any other matplotlib-related import.
We then import from matplotlib a Tk-specialized canvas and navigation toolbar (the latter is optional, used for zomming and tilting).

In [1]:
%gui tk
import tkinter
import numpy as np

import matplotlib as mpl
mpl.use('TkAgg')
print("Matplotlib version:", mpl.__version__)
import matplotlib.pyplot as plt

from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg as FigureCanvas

Matplotlib version: 3.0.0


/Users/yoavram/miniconda3/envs/DataSciPy/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: matplotlib.pyplot as already been imported, this call will have no effect.


In [2]:
class MainWindow:
    def __init__(self):
        self.master = master = tkinter.Tk()
        self.master.title("Tk and Matplotlib")
        self.fig = mpl.figure.Figure(figsize=(6,4), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvas(self.fig, master=master)
        self.canvas.get_tk_widget().pack()

app = MainWindow()

In [3]:
app.ax.plot(range(10))
app.canvas.draw()

Now we can plot to the window from the notebook. Most of this code is the same as we always do with matplottlib, only that:
- we already have `fig` and `ax` objects, defined in the `__init__` above cell
- when we are done plotting we need to call `window.canvas.draw()` to make it update the GUI.

Here's a more complex plot:

In [4]:
x = np.linspace(0, 2 * np.pi, 100)
y1 = np.sin(x)
y2 = np.cos(x)

app.ax.plot(x, y1)
app.ax.plot(x, y2)
app.ax.set(
    xlabel='x',
    ylabel='y',
    xlim=(x.min(), x.max()),
    ylim=(y1.min(), y1.max()),
)
app.fig.tight_layout()
app.canvas.draw()

## Figure events

We'd like to be able to act when the user initiates [events on the figure](http://matplotlib.org/users/event_handling.html). 

We'll start with a simple event: the user clicks on the plot and the app prints the event details to the console:

In [5]:
def on_click(event):
    print(event)

This `onclick` expects a [`MouseEvent`](http://matplotlib.org/api/backend_bases_api.html#matplotlib.backend_bases.MouseEvent) instance that has several attributes:

- `button` is the name of the button (1 for left click, 3 for right click)
- `x`, `y` are the pixels coordinates
- `xdata`, `ydata` are the data coordinates
- `dblclick` if the click was a double click
- `inaxes` the axes in which the click occured; use to change the plot
- `canvas` the canvas in which the click occured; use to update the plot with `canvas.draw()`

We connect this callback/event handler to the canvas object of the figure:

In [6]:
cid = app.fig.canvas.mpl_connect('button_press_event', on_click)

MPL MouseEvent: xy=(314,245) xydata=(2.8866782759331624,0.16262183402653507) button=1 dblclick=False inaxes=AxesSubplot(0.144421,0.145694;0.824745x0.803008)
MPL MouseEvent: xy=(375,254) xydata=(3.6612088512828143,0.21865407020489447) button=1 dblclick=False inaxes=AxesSubplot(0.144421,0.145694;0.824745x0.803008)
MPL MouseEvent: xy=(445,209) xydata=(4.550014429552906,-0.06150711068690318) button=1 dblclick=False inaxes=AxesSubplot(0.144421,0.145694;0.824745x0.803008)
MPL MouseEvent: xy=(302,166) xydata=(2.734311605372575,-0.3292166835390655) button=3 dblclick=False inaxes=AxesSubplot(0.144421,0.145694;0.824745x0.803008)


To disconnect the callback, you can call (you should do this before supplying a different callback):

In [7]:
app.fig.canvas.mpl_disconnect(cid)

## Example

Let's do something more interesting: when the user double clicks the plot, we will add a black circle marker at that point:

In [8]:
def on_click_marker(event):
    if event.dblclick:
        x, y = event.xdata, event.ydata
        ax = event.inaxes
        ax.plot(x, y, 'ko')
        event.canvas.draw()

In [9]:
cid = app.fig.canvas.mpl_connect('button_press_event', on_click_marker)

In [10]:
app.fig.canvas.mpl_disconnect(cid)

## Exercise

We also want that when the user clicks the right button mouse on the plot, a straight line will be plotted from the origin to where the mouse is at.

In [33]:
def on_click_line(event):
    pass

In [34]:
cid = app.fig.canvas.mpl_connect('button_press_event', on_click_line)

# Polygon

In [11]:
class Polygonization:
    def __init__(self):
        self.master = master = tkinter.Tk()
        master.title("Polygonization")
        self.frame = tkinter.Frame(master)
        self.frame.pack(padx=15,pady=15)
        
        # mpl stuff
        self.fig = mpl.figure.Figure(figsize=(6, 4), dpi=100)
        self.ax = self.fig.add_subplot(111)
        self.canvas = FigureCanvas(self.fig, master=self.frame)
        self.canvas.get_tk_widget().pack()
        
        # callbacks
        self.btn_prs_cid = self.fig.canvas.mpl_connect('button_press_event', self.mouse_handler)
        self.btn_rls_cid = self.fig.canvas.mpl_connect('button_release_event', self.mouse_handler)
        self.btn_mtn_cid = self.fig.canvas.mpl_connect('motion_notify_event', self.mouse_handler)
        
        # draw plot
        x = np.linspace(0, 2 * np.pi, 100)
        y = np.sin(x)
        self.ax.plot(x, y)

    
    def mouse_handler(self, event):
        if not event.dblclick and event.button == 1 and event.inaxes:
            ax = event.inaxes
            if event.name == 'button_press_event':
                self.path = ax.plot([], [], 'k-')[0]
            xdata, ydata = self.path.get_data()
            x, y = event.xdata, event.ydata            
            xdata = np.append(xdata, x)
            ydata = np.append(ydata, y)
            if event.name == 'button_release_event':
                xdata = np.append(xdata, xdata[0])
                ydata = np.append(ydata, ydata[0])
            self.path.set_data(xdata, ydata)
            if event.name == 'button_release_event':                
                data = np.array(self.path.get_data())
                self.poly = plt.Polygon(data.T, alpha=0.3)   
                if self.poly.contains_point((0,0)):
                    self.poly.set_color('r')
                ax.add_artist(self.poly)
            self.canvas.draw()

app = Polygonization()

In [12]:
app.path.get_data()

(array([2.18290019, 2.18290019, 2.19776363, 2.21262708, 2.28694433,
        2.43557882, 2.61394021, 2.7031209 , 2.77743815, 2.7923016 ,
        2.80716505, 2.8517554 , 3.22334162, 3.50574716, 3.77328924,
        3.95165063, 3.99624098, 4.01110443, 4.02596788, 4.27864651,
        4.70968654, 5.03668242, 5.22990725, 5.31908795, 5.34881485,
        5.34881485, 5.3636783 , 5.39340519, 5.43799554, 5.45285899,
        5.46772244, 5.46772244, 5.40826864, 5.3042245 , 5.00695552,
        4.69482309, 4.23405616, 3.89219683, 3.5800644 , 3.29765887,
        3.03011679, 2.7625747 , 2.52475952, 2.33153468, 2.21262708,
        1.98967535, 1.75186016, 1.49918153, 1.36541048, 1.26136634,
        1.17218565, 1.14245875, 1.14245875, 1.20191254, 1.30595669,
        1.48431808, 1.76672361, 1.76672361, 2.18290019]),
 array([ 0.36419256,  0.35690871,  0.35690871,  0.37147641,  0.38604412,
         0.42974722,  0.48073418,  0.50258574,  0.51715344,  0.51715344,
         0.51715344,  0.52443729,  0.5827081 ,  

# References

-  [TkDocs](http://www.tkdocs.com/)
- [Tkinter 8.5 reference](http://infohost.nmt.edu/tcc/help/pubs/tkinter/web/index.html)
- [Learning IPython for Interactive Computing and Data Visualization](http://ipython-books.github.io/minibook/) by Cyrille Rossant, pg. 85 (Qt example, but a good tutorial).
- [Pygubu](https://github.com/alejandroautalan/pygubu) - Tkinter designer
- [Embedding Matplotlib plots in Tk applications](http://matplotlib.org/examples/user_interfaces/embedding_in_tk.html)
- [Matplotlib: event handling and picking](http://matplotlib.org/users/event_handling.html)
- Jake Vanderplas's [Minesweeper in Matplotlib](https://jakevdp.github.io/blog/2012/12/06/minesweeper-in-matplotlib/), [Quaternions and Key Bindings: Simple 3D Visualization in Matplotlib](http://jakevdp.github.io/blog/2012/11/24/simple-3d-visualization-in-matplotlib/) and [3D Interactive Rubik's Cube in Python](http://jakevdp.github.io/blog/2012/11/26/3d-interactive-rubiks-cube-in-python/) are amazing examples of what can be done with Matplotlib beyond simple plots and using a GUI.

## Colophon
This notebook was written by [Yoav Ram](http://python.yoavram.com) and is part of the [_Python for Engineers_](https://github.com/yoavram/Py4Eng) course.

The notebook was written using [Python](http://python.org/) 3.6.1.
Dependencies listed in [environment.yml](../environment.yml), full versions in [environment_full.yml](../environment_full.yml).

This work is licensed under a CC BY-NC-SA 4.0 International License.

![Python logo](https://www.python.org/static/community_logos/python-logo.png)